In [75]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [76]:
!pip install recmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [77]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from surprise import SVD, Dataset, Reader, KNNBaseline,accuracy
from surprise.model_selection import train_test_split
import recmetrics

In [78]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [79]:
ratings=pd.read_csv('/content/drive/Shareddrives/rs assignment1/interactions_train.csv')
# Take a look at the data
ratings.head()

,user_id,recipe_id,date,rating,u,i
0,2046,4684,2000-02-25,5.0,22095,44367
1,2046,517,2000-02-25,5.0,22095,87844
2,1773,7435,2000-03-13,5.0,24732,138181
3,1773,278,2000-03-13,4.0,24732,93054
4,2046,3431,2000-04-07,5.0,22095,101723


In [80]:
# Read in data
recipes = pd.read_csv('/content/drive/Shareddrives/rs assignment1/RAW_recipes.csv')
# Take a look at the data
recipes.head()
recipes.rename(columns = {'id':'recipe_id'}, inplace = True)

In [81]:

ratings = ratings.sample(frac = 0.002)

In [82]:
reader = Reader(rating_scale=(0, 5))
data = Dataset.load_from_df(ratings[['user_id', 'recipe_id', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=0.25)

In [83]:
def recommendation(algo, trainset, testset):
  # Train the algorithm on the trainset, and predict ratings for the testset
  algo.fit(trainset)

  # Predictions on testing set
  test_predictions = algo.test(testset)
  test_rmse = accuracy.rmse(test_predictions)
  test_mae = accuracy.mae(test_predictions)
  
  return test_rmse, test_mae, test_predictions

In [84]:
sim_options = {'name': 'pearson_baseline', 'user_based': False}
algo = KNNBaseline()
test_knn_rmse, test_knn_mae, test_knn_pred = recommendation(algo, trainset, testset)
     

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.8636
MAE:  0.6028


In [85]:
algo = SVD()
test_svd_rmse, test_svd_mae, test_svd_pred  = recommendation(algo, trainset, testset)

RMSE: 0.8626
MAE:  0.6013


In [86]:
test_pred_df = pd.DataFrame(columns= ['user_id', 'recipe_id', 'actual', 'svd_rating', 'knn_rating'])

In [87]:
num_test = len(test_svd_pred)
print(num_test)
     

350


In [88]:
for i in range(num_test): 
  svd = test_svd_pred[i]
  
  knn = test_knn_pred[i]
  
  df = pd.DataFrame([[svd.uid, svd.iid, svd.r_ui, svd.est, knn.est]], columns=['user_id', 'recipe_id', 'actual', 'svd_rating', 'knn_rating'])
  # print(df)
  test_pred_df = pd.concat([df, test_pred_df], ignore_index=True)

In [89]:
test_pred_df

,user_id,recipe_id,actual,svd_rating,knn_rating
0,103876,223103,5.0,4.602532,4.596606
1,293410,284500,5.0,4.547710,4.547710
2,188119,123454,5.0,4.547710,4.547710
3,255186,60284,5.0,4.547710,4.547710
4,47510,153556,5.0,4.581187,4.573555
...,...,...,...,...,...
345,160974,299184,5.0,4.692035,4.635962
346,85018,232520,5.0,4.547710,4.547710
347,335277,197429,5.0,4.547710,4.547710
348,195175,84614,5.0,4.599404,4.571515


In [90]:
#create model (matrix of predicted values)
svd_model = test_pred_df.pivot_table(index='user_id', columns='recipe_id', values = 'svd_rating').fillna(0)
knn_model = test_pred_df.pivot_table(index='user_id', columns='recipe_id', values = 'knn_rating').fillna(0)

In [91]:
def get_users_predictions(user_id, n, model):
    recommended_items = pd.DataFrame(model.loc[user_id])
    recommended_items.columns = ["predicted_rating"]
    recommended_items = recommended_items.sort_values('predicted_rating', ascending=False)    
    recommended_items = recommended_items.head(n)
    return recommended_items.index.tolist()

In [92]:
test = test_pred_df.copy().groupby('user_id', as_index=False)['recipe_id'].agg({'actual': (lambda x: list(set(x)))})

In [93]:
test = test.set_index("user_id")

In [94]:
# make recommendations for all members in the test data
svd_recs = [] = []
for user in test.index:
    svd_predictions = get_users_predictions(user, 10, svd_model)
    svd_recs.append(svd_predictions)
        
test['svd_predictions'] = svd_recs
test.head()

,actual,svd_predictions
user_id,,
1535,[208012],"[208012, 191067, 204409, 203275, 202177, 19847..."
2310,[75265],"[75265, 191067, 204409, 203275, 202177, 198470..."
4439,"[262651, 123999]","[262651, 123999, 191067, 204409, 203275, 20217..."
4470,"[49353, 159554]","[159554, 49353, 206969, 204409, 203275, 202177..."
5060,"[202177, 17542, 384653, 76430, 74326, 16921]","[76430, 384653, 17542, 16921, 202177, 74326, 2..."


In [95]:
# make recommendations for all members in the test data
knn_recs = [] = []
for user in test.index:
    knn_predictions = get_users_predictions(user, 10, knn_model)
    knn_recs.append(knn_predictions)
        
test['knn_predictions'] = knn_recs
test.head()

,actual,svd_predictions,knn_predictions
user_id,,,
1535,[208012],"[208012, 191067, 204409, 203275, 202177, 19847...","[208012, 191067, 204409, 203275, 202177, 19847..."
2310,[75265],"[75265, 191067, 204409, 203275, 202177, 198470...","[75265, 191067, 204409, 203275, 202177, 198470..."
4439,"[262651, 123999]","[262651, 123999, 191067, 204409, 203275, 20217...","[262651, 123999, 191067, 204409, 203275, 20217..."
4470,"[49353, 159554]","[159554, 49353, 206969, 204409, 203275, 202177...","[159554, 49353, 206969, 204409, 203275, 202177..."
5060,"[202177, 17542, 384653, 76430, 74326, 16921]","[76430, 384653, 17542, 16921, 202177, 74326, 2...","[76430, 384653, 17542, 16921, 202177, 74326, 2..."


In [98]:
users = ratings["user_id"].value_counts()
nov = ratings.recipe_id.value_counts()
pop = dict(nov)
svd_novelty,svd_mselfinfo_list = recmetrics.novelty(svd_recs, pop, len(users), 10)
print(svd_novelty)

10.015124605702105


In [99]:
knn_novelty,knn_mselfinfo_list = recmetrics.novelty(knn_recs, pop, len(users), 10)
print(knn_novelty)

10.015124605702105


In [102]:
catalog = ratings.recipe_id.unique().tolist()
svd_cat_coverage = recmetrics.catalog_coverage(svd_recs, catalog, 100)
print(svd_cat_coverage)

8.04


In [103]:
knn_cat_coverage = recmetrics.catalog_coverage(knn_recs, catalog, 100)
print(knn_cat_coverage)

8.19
